COPYING TYLERE'S EXAMPLE: https://groups.google.com/d/msg/google-earth-engine-developers/ZEN--ZzVyBE/MWy1acrHBAAJ

Import modules

In [15]:
import ee
import pandas as pd

In [16]:
ee.Initialize()

Create geometry

In [17]:
geometry = ee.Geometry.Polygon(
        [[[-120.7342529296875, 36.94769679250732],
          [-120.76995849609375, 36.88621127842176],
          [-120.57769775390625, 36.83346996591303],
          [-120.56671142578125, 36.88840804313823]]])

Create Feature collection

In [18]:
ft = ee.FeatureCollection('USGS/WBD/2017/HUC12').filterBounds(geometry)

Calculate area

In [19]:
hectares = ee.Image.pixelArea()

Use Feature Collection as a reducer

In [20]:
ret = hectares.reduceRegions(ft, 'sum', 30)

Define maximum number of features to return

In [21]:
MAX_FEATURES=100

Create function to return information on features

In [22]:
def get_feature_sum(feat):
  return ee.Feature(feat).get('sum')

Call function

In [23]:
print(ret.toList(MAX_FEATURES).map(get_feature_sum).getInfo())

[349166661.4217493, 160245915.6243403, 189242055.62129444]


Link to Google Earth Engine Developers list: 
    https://groups.google.com/d/msg/google-earth-engine-developers/wbeU0JlvgAQ/CBTaAwCWGAAJ

Import states

In [24]:
state_list = ['Iowa','Illinois']

In [25]:
REGION_COLLECTION = ee.FeatureCollection('ft:1fRY18cjsHzDgGiJiS2nnpUU3v9JPDc2HNaR7Xk8')
STATE_COLLECTION = REGION_COLLECTION.filter(ee.Filter.inList('Name', state_list))

Create function reducer using FeatureCollection

In [26]:
def calc_img_mean(img_coll):
    # get image values
    reduction = img_coll.reduceRegions(STATE_COLLECTION, ee.Reducer.mean())
    # get list of country names
    ids = reduction.reduceColumns(ee.Reducer.toList(),['Name']).get('list')
    # get list of means
    means = reduction.reduceColumns(ee.Reducer.toList(),['mean']).get('list')
    # add time and return
    feature_dict = ee.Feature(None,ee.Dictionary.fromLists(ids, means).set('time', img_coll.get('system:time_start')))
    return feature_dict

In [27]:
palmer_collection = ee.ImageCollection('IDAHO_EPSCOR/PDSI')

In [28]:
data = ee.FeatureCollection(palmer_collection.map(calc_img_mean, opt_dropNulls=True))

In [14]:
# csv_url = data.getDownloadURL()
# df = pd.read_csv(csv_url)

NameError: name 'pd' is not defined

In [29]:
# taskid = ee.data.newTaskId(1)[0]

# batch_test = ee.batch.data.startProcessing(taskid,data)
task = ee.batch.Export.table.toDrive({collection: table, description: 'filename'})
task.start()

NameError: name 'table' is not defined